Flusense

In [ ]:

# Extracting cough and sneeze audios from Flusense dataset
import os
import librosa
import soundfile as sf
from textgrid import TextGrid

# Paths
textgrid_dir = r"C:\Users\DELL\FluSense-data\flusense_data"
audio_dirs = [
    r"D:\Project\Data\ZIP FILES\Flusense\FluSense audio-20250416T034942Z-001\FluSense audio",
    r"D:\Project\Data\ZIP FILES\Flusense\FluSense audio-20250416T034942Z-002\FluSense audio",
    r"D:\Project\Data\ZIP FILES\Flusense\FluSense audio-20250416T034942Z-003\FluSense audio",
    r"D:\Project\Data\ZIP FILES\Flusense\FluSense audio-20250416T034942Z-004\FluSense audio"
]
output_cough = r"D:\Project\Data\segmented_cough"
output_sneeze = r"D:\Project\Data\segmented_sneeze"
os.makedirs(output_cough, exist_ok=True)
os.makedirs(output_sneeze, exist_ok=True)

sample_rate = 16000

def find_audio(file_base):
    for d in audio_dirs:
        audio_path = os.path.join(d, file_base + ".wav")
        if os.path.exists(audio_path):
            return audio_path
    return None

cough_count = 0
sneeze_count = 0
skipped = 0

for tg_file in os.listdir(textgrid_dir):
    if tg_file.endswith(".TextGrid"):
        base = os.path.splitext(tg_file)[0]
        audio_path = find_audio(base)
        if not audio_path:
            continue

        tg = TextGrid.fromFile(os.path.join(textgrid_dir, tg_file))

        # Search for common tier names
        tier = None
        for name in ["label", "labels", "word", "words", "event", "events"]:
            tier = next((t for t in tg.tiers if t.name.lower() == name), None)
            if tier:
                break
        if not tier:
            continue

        try:
            y, sr = librosa.load(audio_path, sr=sample_rate)
        except Exception as e:
            print(f"Error loading {audio_path}: {e}")
            continue

        for i, interval in enumerate(tier.intervals):
            label = interval.mark.strip().lower()
            if label in ["cough", "sneeze"]:
                start = int(interval.minTime * sr)
                end = int(interval.maxTime * sr)
                if end <= start or end > len(y):
                    skipped += 1
                    continue

                clip = y[start:end]
                out_dir = output_cough if label == "cough" else output_sneeze
                file_name = f"{base}_{label}_{i}.wav"
                sf.write(os.path.join(out_dir, file_name), clip, sr)

                if label == "cough":
                    cough_count += 1
                else:
                    sneeze_count += 1

print(f"\n Total cough segments extracted: {cough_count}")
print(f"Total sneeze segments extracted: {sneeze_count}")
print(f" Segments skipped due to timing issues: {skipped}")


In [ ]:
#splitting 2 sc, mono , 16KHZ
from pydub import AudioSegment
import os

input_folder = r"D:\Project\Data\segmented_sneeze"
output_folder = r"D:\Project\Data\Processed\Flusense\sneeze"
os.makedirs(output_folder, exist_ok=True)

for file_name in os.listdir(input_folder):
    if file_name.endswith((".wav", ".mp3", ".flac")):
        sound = AudioSegment.from_file(os.path.join(input_folder, file_name))
        sound = sound.set_channels(1).set_frame_rate(16000)

        total_ms = len(sound)
        base = os.path.splitext(file_name)[0]

        for i in range(0, total_ms, 2000):
            chunk = sound[i:i+2000]
            if len(chunk) < 2000:
                # Pad short chunks with silence
                padding = AudioSegment.silent(duration=2000 - len(chunk))
                chunk = chunk + padding

            output_path = os.path.join(output_folder, f"{base}_part{i//2000}.wav")
            chunk.export(output_path, format="wav")

print("All segments (including short ones) saved.")

In [ ]:
# Removing silent audios 
from pydub import AudioSegment
import os

processed_folder = r"D:\Project\Data\Processed\Flusense\sneeze"
threshold_dBFS = -40  # threshold for silence (you can tune this)

for file_name in os.listdir(processed_folder):
    if file_name.endswith(".wav"):
        path = os.path.join(processed_folder, file_name)
        audio = AudioSegment.from_file(path)

        if audio.dBFS < threshold_dBFS:
            print(f"Deleting {file_name} (too quiet: {audio.dBFS:.2f} dBFS)")
            os.remove(path)

In [ ]:
#splitting 2 sc, mono , 16KHZ -cough
from pydub import AudioSegment
import os

input_folder = r"D:\Project\Data\segmented_cough"
output_folder = r"D:\Project\Data\Processed\temp\Flusense"
os.makedirs(output_folder, exist_ok=True)

for file_name in os.listdir(input_folder):
    if file_name.endswith((".wav", ".mp3", ".flac")):
        sound = AudioSegment.from_file(os.path.join(input_folder, file_name))
        sound = sound.set_channels(1).set_frame_rate(16000)

        total_ms = len(sound)
        base = os.path.splitext(file_name)[0]

        for i in range(0, total_ms, 2000):
            chunk = sound[i:i+2000]
            if len(chunk) < 2000:
                # Pad short chunks with silence
                padding = AudioSegment.silent(duration=2000 - len(chunk))
                chunk = chunk + padding

            output_path = os.path.join(output_folder, f"{base}_part{i//2000}.wav")
            chunk.export(output_path, format="wav")

print("All segments (including short ones) saved.")


In [ ]:
# Removing silent audios 
from pydub import AudioSegment
import os

processed_folder = r"D:\Project\Data\Processed\temp\Flusense"
threshold_dBFS = -40  # threshold for silence (you can tune this)

for file_name in os.listdir(processed_folder):
    if file_name.endswith(".wav"):
        path = os.path.join(processed_folder, file_name)
        audio = AudioSegment.from_file(path)

        if audio.dBFS < threshold_dBFS:
            print(f"Deleting {file_name} (too quiet: {audio.dBFS:.2f} dBFS)")
            os.remove(path)

In [ ]:
# no_event extraction
import os
import librosa
import soundfile as sf
from textgrid import TextGrid

# Paths
textgrid_dir = r"C:\Users\DELL\FluSense-data\flusense_data"
audio_dirs = [
    r"D:\Project\Data\ZIP FILES\Flusense\FluSense audio-20250416T034942Z-001\FluSense audio",
    r"D:\Project\Data\ZIP FILES\Flusense\FluSense audio-20250416T034942Z-002\FluSense audio",
    r"D:\Project\Data\ZIP FILES\Flusense\FluSense audio-20250416T034942Z-003\FluSense audio",
    r"D:\Project\Data\ZIP FILES\Flusense\FluSense audio-20250416T034942Z-004\FluSense audio"
]

output_no_event = r"D:\Project\Data\segmented_no_event"
os.makedirs(output_no_event, exist_ok=True)

sample_rate = 16000
no_event_count = 0
skipped = 0

# Look up audio path by base filename
def find_audio(file_base):
    for d in audio_dirs:
        audio_path = os.path.join(d, file_base + ".wav")
        if os.path.exists(audio_path):
            return audio_path
    return None

# Extract no_event (speech, silence, etc)
for tg_file in os.listdir(textgrid_dir):
    if not tg_file.endswith(".TextGrid"):
        continue

    base = os.path.splitext(tg_file)[0]
    audio_path = find_audio(base)
    if not audio_path:
        continue

    tg = TextGrid.fromFile(os.path.join(textgrid_dir, tg_file))

    tier = None
    for name in ["label", "labels", "word", "words", "event", "events"]:
        tier = next((t for t in tg.tiers if t.name.lower() == name), None)
        if tier:
            break
    if not tier:
        continue

    try:
        y, sr = librosa.load(audio_path, sr=sample_rate)
    except Exception as e:
        print(f"Error loading {audio_path}: {e}")
        continue

    for i, interval in enumerate(tier.intervals):
        label = interval.mark.strip().lower()
        if label not in ["speech", "silence", "etc"]:
            continue

        start = int(interval.minTime * sr)
        end = int(interval.maxTime * sr)
        if end <= start or end > len(y):
            skipped += 1
            continue

        clip = y[start:end]
        file_name = f"{base}_{label}_{i}.wav"
        sf.write(os.path.join(output_no_event, file_name), clip, sr)
        no_event_count += 1

print(f"\n Total no_event segments extracted: {no_event_count}")
print(f" Segments skipped due to timing issues: {skipped}")


In [ ]:
#splitting 2 sc, mono , 16KHZ
from pydub import AudioSegment
import os

input_folder = r"D:\Project\Data\segmented_no_event"
output_folder = r"D:\Project\Data\Processed\temp\no_event_2s"
os.makedirs(output_folder, exist_ok=True)

for file_name in os.listdir(input_folder):
    if file_name.endswith((".wav", ".mp3", ".flac")):
        sound = AudioSegment.from_file(os.path.join(input_folder, file_name))
        sound = sound.set_channels(1).set_frame_rate(16000)

        total_ms = len(sound)
        base = os.path.splitext(file_name)[0]

        for i in range(0, total_ms, 2000):
            chunk = sound[i:i+2000]
            if len(chunk) < 2000:
                # Pad short chunks with silence
                padding = AudioSegment.silent(duration=2000 - len(chunk))
                chunk = chunk + padding

            output_path = os.path.join(output_folder, f"{base}_part{i//2000}.wav")
            chunk.export(output_path, format="wav")

print("All segments (including short ones) saved.")


In [ ]:
# to pick randome 12000 clips 
import os
import random
import shutil

# Original dataset path (source of no_event audio clips)
input_dir = r'D:\Project\Data\Processed\temp\no_event_2s'

# Output directory for the balanced subset of 6000 samples
output_dir = r'D:\Project\Data\Processed\temp\sneezeno_event_balanced\no_event_12000'

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# List all relevant .wav files starting with "no_event"
no_event_files = [f for f in os.listdir(input_dir) if f.lower().startswith("no_event") and f.lower().endswith(".wav")]
random.shuffle(no_event_files)

# Copy first 12000 files to the new directory
for file in no_event_files[:12000]:
    shutil.copy(os.path.join(input_dir, file), os.path.join(output_dir, file))

FSD50K

In [ ]:
# Extracting cough audios from the FSD50K dataset
import pandas as pd
import shutil
import os

# Paths
metadata_csv = r"D:\Project\Data\ZIP FILES\FSD50K\FSD50K.metadata\FSD50K.metadata\collection\collection_dev.csv"
audio_folder = r"D:\Project\Data\ZIP FILES\FSD50K\FSD50K.dev_audio"
output_folder = r"D:\Project\Data\Processed\FSD50K_cough_dev"

os.makedirs(output_folder, exist_ok=True)

# Load metadata
df = pd.read_csv(metadata_csv)

# Filter for rows where 'labels' is exactly 'cough'
cough_only = df[df['labels'] == 'Cough']

# Copy corresponding audio files
for fname in cough_only['fname']:
    src = os.path.join(audio_folder, f"{fname}.wav")
    dst = os.path.join(output_folder, f"{fname}.wav")
    if os.path.exists(src):
        shutil.copy(src, dst)

print(f"Copied {len(cough_only)} Cough-only audio files to: {output_folder}")

In [ ]:
#FSD50K splitting 2 sc, mono , 16KHZ -cough
from pydub import AudioSegment
import os

input_folder = r"D:\Project\Data\Processed\FSD50K_cough_dev"
output_folder = r"D:\Project\Data\Processed\temp\FSD50K_cough_2s_dev"
os.makedirs(output_folder, exist_ok=True)

for file_name in os.listdir(input_folder):
    if file_name.endswith((".wav", ".mp3", ".flac")):
        sound = AudioSegment.from_file(os.path.join(input_folder, file_name))
        sound = sound.set_channels(1).set_frame_rate(16000)

        total_ms = len(sound)
        base = os.path.splitext(file_name)[0]

        for i in range(0, total_ms, 2000):
            chunk = sound[i:i+2000]
            if len(chunk) < 2000:
                # Pad short chunks with silence
                padding = AudioSegment.silent(duration=2000 - len(chunk))
                chunk = chunk + padding

            output_path = os.path.join(output_folder, f"{base}_part{i//2000}.wav")
            chunk.export(output_path, format="wav")

print("All segments (including short ones) saved.")


In [ ]:
# Removing silent audios FSD50K -cough
from pydub import AudioSegment
import os

processed_folder = r"D:\Project\Data\Processed\temp\FSD50K_cough_2s_dev"
threshold_dBFS = -40  # threshold for silence (you can tune this)

for file_name in os.listdir(processed_folder):
    if file_name.endswith(".wav"):
        path = os.path.join(processed_folder, file_name)
        audio = AudioSegment.from_file(path)

        if audio.dBFS < threshold_dBFS:
            print(f"Deleting {file_name} (too quiet: {audio.dBFS:.2f} dBFS)")
            os.remove(path)


In [ ]:
# Extracting sneeze audios from the FSD50K dataset
import pandas as pd
import shutil
import os

# Paths
metadata_csv = r"D:\Project\Data\ZIP FILES\FSD50K\FSD50K.metadata\FSD50K.metadata\collection\collection_dev.csv"
audio_folder = r"D:\Project\Data\ZIP FILES\FSD50K\FSD50K.dev_audio"
output_folder = r"D:\Project\Data\Processed\FSD50K_sneeze_dev"

os.makedirs(output_folder, exist_ok=True)

# Load metadata
df = pd.read_csv(metadata_csv)

# Filter for rows where 'labels' is exactly 'Sneeze'
sneeze_only = df[df['labels'] == 'Sneeze']

# Copy corresponding audio files
for fname in sneeze_only['fname']:
    src = os.path.join(audio_folder, f"{fname}.wav")
    dst = os.path.join(output_folder, f"{fname}.wav")
    if os.path.exists(src):
        shutil.copy(src, dst)

print(f"Copied {len(sneeze_only)} Sneeze-only audio files to: {output_folder}")

In [ ]:
#FSD50K splitting 2 sc, mono , 16KHZ - sneeze
from pydub import AudioSegment
import os

input_folder = r"D:\Project\Data\Processed\FSD50K_sneeze_dev"
output_folder = r"D:\Project\Data\Processed\temp\FSD50K_sneeze_2s_dev"
os.makedirs(output_folder, exist_ok=True)

for file_name in os.listdir(input_folder):
    if file_name.endswith((".wav", ".mp3", ".flac")):
        sound = AudioSegment.from_file(os.path.join(input_folder, file_name))
        sound = sound.set_channels(1).set_frame_rate(16000)

        total_ms = len(sound)
        base = os.path.splitext(file_name)[0]

        for i in range(0, total_ms, 2000):
            chunk = sound[i:i+2000]
            if len(chunk) < 2000:
                # Pad short chunks with silence
                padding = AudioSegment.silent(duration=2000 - len(chunk))
                chunk = chunk + padding

            output_path = os.path.join(output_folder, f"{base}_part{i//2000}.wav")
            chunk.export(output_path, format="wav")

print("All segments (including short ones) saved.")


In [ ]:
# Removing silent audios FSD50K - sneeze
from pydub import AudioSegment
import os

processed_folder = r"D:\Project\Data\Processed\temp\FSD50K_sneeze_2s_dev"
threshold_dBFS = -40  # threshold for silence (you can tune this)

for file_name in os.listdir(processed_folder):
    if file_name.endswith(".wav"):
        path = os.path.join(processed_folder, file_name)
        audio = AudioSegment.from_file(path)

        if audio.dBFS < threshold_dBFS:
            print(f"Deleting {file_name} (too quiet: {audio.dBFS:.2f} dBFS)")
            os.remove(path)


ECS50

In [ ]:
import os
import pandas as pd
import shutil

# Step 1: Define paths
meta_csv = r"D:\Project\Data\ZIP FILES\ESC-50-master\ESC-50-master\meta\esc50.csv"
audio_dir = r"D:\Project\Data\ZIP FILES\ESC-50-master\ESC-50-master\audio"
output_dir = r"D:\Project\Data\Processed\temp\cough_esc50"
os.makedirs(output_dir, exist_ok=True)

# Step 2: Load metadata
df = pd.read_csv(meta_csv)

# Step 3: Filter for 'sneeze' category
sneeze_files = df[df["category"] == "coughing"]["filename"]

# Step 4: Copy matching files
copied = 0
for file in sneeze_files:
    src = os.path.join(audio_dir, file)
    dst = os.path.join(output_dir, file)
    if os.path.exists(src):
        shutil.copy(src, dst)
        copied += 1
    else:
        print(f" File not found: {file}")

print(f"\n Total sneeze clips copied: {copied}")


In [ ]:
#splitting 2 sc, mono , 16KHZ
from pydub import AudioSegment
import os

input_folder = r"D:\Project\Data\Processed\temp\cough_esc50"
output_folder = r"D:\Project\Data\Processed\temp\cough_esc50_2s"
os.makedirs(output_folder, exist_ok=True)

for file_name in os.listdir(input_folder):
    if file_name.endswith((".wav", ".mp3", ".flac")):
        sound = AudioSegment.from_file(os.path.join(input_folder, file_name))
        sound = sound.set_channels(1).set_frame_rate(16000)

        total_ms = len(sound)
        base = os.path.splitext(file_name)[0]

        for i in range(0, total_ms, 2000):
            chunk = sound[i:i+2000]
            if len(chunk) < 2000:
                # Pad short chunks with silence
                padding = AudioSegment.silent(duration=2000 - len(chunk))
                chunk = chunk + padding

            output_path = os.path.join(output_folder, f"{base}_part{i//2000}.wav")
            chunk.export(output_path, format="wav")

print("All segments (including short ones) saved.")


In [ ]:
# Removing silent audios 
from pydub import AudioSegment
import os

processed_folder = r"D:\Project\Data\Processed\temp\cough_esc50_2s"
threshold_dBFS = -40  # threshold for silence (you can tune this)

for file_name in os.listdir(processed_folder):
    if file_name.endswith(".wav"):
        path = os.path.join(processed_folder, file_name)
        audio = AudioSegment.from_file(path)

        if audio.dBFS < threshold_dBFS:
            print(f"Deleting {file_name} (too quiet: {audio.dBFS:.2f} dBFS)")
            os.remove(path)

In [ ]:
import os
import pandas as pd
import shutil

# Step 1: Define paths
meta_csv = r"D:\Project\Data\ZIP FILES\ESC-50-master\ESC-50-master\meta\esc50.csv"
audio_dir = r"D:\Project\Data\ZIP FILES\ESC-50-master\ESC-50-master\audio"
output_dir = r"D:\Project\Data\Processed\temp\sneeze_esc50"
os.makedirs(output_dir, exist_ok=True)

# Step 2: Load metadata
df = pd.read_csv(meta_csv)

# Step 3: Filter for 'sneeze' category
sneeze_files = df[df["category"] == "sneezing"]["filename"]

# Step 4: Copy matching files
copied = 0
for file in sneeze_files:
    src = os.path.join(audio_dir, file)
    dst = os.path.join(output_dir, file)
    if os.path.exists(src):
        shutil.copy(src, dst)
        copied += 1
    else:
        print(f" File not found: {file}")

print(f"\n Total sneeze clips copied: {copied}")


In [ ]:
#splitting 2 sc, mono , 16KHZ
from pydub import AudioSegment
import os

input_folder = r"D:\Project\Data\Processed\sneeze\esc50"
output_folder = r"D:\Project\Data\Processed\sneeze\esc50_clean"
os.makedirs(output_folder, exist_ok=True)

for file_name in os.listdir(input_folder):
    if file_name.endswith((".wav", ".mp3", ".flac")):
        sound = AudioSegment.from_file(os.path.join(input_folder, file_name))
        sound = sound.set_channels(1).set_frame_rate(16000)

        total_ms = len(sound)
        base = os.path.splitext(file_name)[0]

        for i in range(0, total_ms, 2000):
            chunk = sound[i:i+2000]
            if len(chunk) < 2000:
                # Pad short chunks with silence
                padding = AudioSegment.silent(duration=2000 - len(chunk))
                chunk = chunk + padding

            output_path = os.path.join(output_folder, f"{base}_part{i//2000}.wav")
            chunk.export(output_path, format="wav")

print("All segments (including short ones) saved.")


In [ ]:
# Removing silent audios 
from pydub import AudioSegment
import os

processed_folder = r"D:\Project\Data\Processed\sneeze\esc50_clean"
threshold_dBFS = -40  # threshold for silence (you can tune this)

for file_name in os.listdir(processed_folder):
    if file_name.endswith(".wav"):
        path = os.path.join(processed_folder, file_name)
        audio = AudioSegment.from_file(path)

        if audio.dBFS < threshold_dBFS:
            print(f"Deleting {file_name} (too quiet: {audio.dBFS:.2f} dBFS)")
            os.remove(path)

NonverbalVocalization

In [ ]:
#NonverbalVocalization splitting 2 sc, mono , 16KHZ
from pydub import AudioSegment
import os

input_folder = r"D:\Project\Data\ZIP FILES\NonverbalVocalization\sneezing"
output_folder = r"D:\Project\Data\Processed\NonverbalVocalization\sneezing_2s"
os.makedirs(output_folder, exist_ok=True)

for file_name in os.listdir(input_folder):
    if file_name.endswith((".wav", ".mp3", ".flac")):
        sound = AudioSegment.from_file(os.path.join(input_folder, file_name))
        sound = sound.set_channels(1).set_frame_rate(16000)

        total_ms = len(sound)
        base = os.path.splitext(file_name)[0]

        for i in range(0, total_ms, 2000):
            chunk = sound[i:i+2000]
            if len(chunk) < 2000:
                # Pad short chunks with silence
                padding = AudioSegment.silent(duration=2000 - len(chunk))
                chunk = chunk + padding

            output_path = os.path.join(output_folder, f"{base}_part{i//2000}.wav")
            chunk.export(output_path, format="wav")

print("All segments (including short ones) saved.")

In [ ]:
# Removing silent audios NonverbalVocalization 
from pydub import AudioSegment
import os

processed_folder = r"D:\Project\Data\Processed\NonverbalVocalization\sneezing_2s"
threshold_dBFS = -40  # threshold for silence (you can tune this)

for file_name in os.listdir(processed_folder):
    if file_name.endswith(".wav"):
        path = os.path.join(processed_folder, file_name)
        audio = AudioSegment.from_file(path)

        if audio.dBFS < threshold_dBFS:
            print(f"Deleting {file_name} (too quiet: {audio.dBFS:.2f} dBFS)")
            os.remove(path)

Pixabay

In [ ]:
#D:\Project\Data\sneeze_pixabay
from pydub import AudioSegment
import os

# Input and output folders
mp3_dir = r"D:\Project\Data\sneeze_pixabay"
wav_dir = r"D:\Project\Data\sneeze_pixabay_wav"
os.makedirs(wav_dir, exist_ok=True)

# Loop through all MP3 files
for file in os.listdir(mp3_dir):
    if file.endswith(".mp3"):
        mp3_path = os.path.join(mp3_dir, file)
        wav_path = os.path.join(wav_dir, file.replace(".mp3", ".wav"))

        try:
            audio = AudioSegment.from_mp3(mp3_path)
            audio = audio.set_channels(1).set_frame_rate(16000)  # Mono, 16 kHz
            audio.export(wav_path, format="wav")
            print(f"Converted: {file}")
        except Exception as e:
            print(f" Failed to convert {file}: {e}")


In [ ]:
import os
import librosa
import soundfile as sf

# Input folder with long sneeze .wav files
input_dir = r"D:\Project\Data\sneeze_pixabay_wav"

# Output folder for 2-second slices
output_dir = r"D:\Project\Data\sneeze_pixabay_sliced"
os.makedirs(output_dir, exist_ok=True)

# Slice config
clip_duration = 2  # seconds
sample_rate = 16000
clip_samples = clip_duration * sample_rate

# Go through each .wav file
for file in os.listdir(input_dir):
    if not file.endswith(".wav"):
        continue

    input_path = os.path.join(input_dir, file)
    y, sr = librosa.load(input_path, sr=sample_rate, mono=True)

    # Slice into fixed-size chunks
    for i in range(0, len(y), clip_samples):
        chunk = y[i:i + clip_samples]
        if len(chunk) < clip_samples:
            continue  # skip short tail clips

        out_name = f"{os.path.splitext(file)[0]}_slice_{i // clip_samples}.wav"
        out_path = os.path.join(output_dir, out_name)
        sf.write(out_path, chunk, sample_rate)

print(f"\nSlicing complete. 2s clips saved to: {output_dir}")


In [ ]:
# Removing silent audios 
from pydub import AudioSegment
import os

processed_folder = r"D:\Project\Data\sneeze_pixabay_sliced"
threshold_dBFS = -40  # threshold for silence (you can tune this)

for file_name in os.listdir(processed_folder):
    if file_name.endswith(".wav"):
        path = os.path.join(processed_folder, file_name)
        audio = AudioSegment.from_file(path)

        if audio.dBFS < threshold_dBFS:
            print(f"Deleting {file_name} (too quiet: {audio.dBFS:.2f} dBFS)")
            os.remove(path)